In [5]:
import tensorflow as tf
import tensorflow_hub as hub

yamnet_model_url = "https://tfhub.dev/google/yamnet/1"
yamnet_model = hub.load(yamnet_model_url)



class YAMNetWrapper(tf.keras.layers.Layer):
    def __init__(self, yamnet_model, **kwargs):
        super().__init__(**kwargs)
        self.yamnet_model = yamnet_model

    def call(self, inputs):
        return self.yamnet_model(inputs)
    
# Função para converter o modelo
def yamnet_tflite(input_shape, yamnet_model):
    input_layer = tf.keras.layers.Input(shape=input_shape, dtype=tf.float32)
    yamnet_wrapper = YAMNetWrapper(yamnet_model)
    yamnet_output = yamnet_wrapper(input_layer)
    model = tf.keras.Model(inputs=input_layer, outputs=yamnet_output)
    return model

# Especifique a forma da entrada
input_shape = (None,)
yamnet_lite_model = yamnet_tflite(input_shape, yamnet_model)

# Converta para TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(yamnet_lite_model)
tflite_model = converter.convert()

ValueError: Exception encountered when calling layer "yam_net_wrapper_2" (type YAMNetWrapper).

in user code:

    File "C:\Users\burim.UNICORP\AppData\Local\Temp\ipykernel_10376\3431620383.py", line 15, in call  *
        return self.yamnet_model(inputs)

    ValueError: Python inputs incompatible with input_signature:
      inputs: (
        Tensor("Placeholder:0", shape=(None, None), dtype=float32))
      input_signature: (
        TensorSpec(shape=(None,), dtype=tf.float32, name=None)).


Call arguments received by layer "yam_net_wrapper_2" (type YAMNetWrapper):
  • inputs=tf.Tensor(shape=(None, None), dtype=float32)

In [9]:
import os
import tempfile

yamnet_saved_model_path = os.path.join(tempfile.gettempdir(), "yamnet_saved_model")
tf.saved_model.save(yamnet_model, yamnet_saved_model_path)






INFO:tensorflow:Assets written to: C:\Users\BURIM~1.UNI\AppData\Local\Temp\yamnet_saved_model\assets


INFO:tensorflow:Assets written to: C:\Users\BURIM~1.UNI\AppData\Local\Temp\yamnet_saved_model\assets


ValueError: This function takes in a list of ConcreteFunction.

In [13]:
loaded_yamnet = tf.saved_model.load(yamnet_saved_model_path)

# Defina a função de assinatura concreta
concrete_func = yamnet_model.signatures['serving_default']

# Converta para TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_model = converter.convert()

with open('yamnet.tflite', 'wb') as f:
    f.write(tflite_model)

In [7]:
print(yamnet_model.signatures)


_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(waveform) at 0x1F3FC1C3490>})
